In [ ]:
#### Code to find coordinates showing video for Detour test - shows the certainty of classification and creates a track image

import cv2
import sys 
import math
import csv
import pandas as pd
import os
from shapely.geometry import Polygon, Point
import numpy as np
import shutil
import time

import object_detection
from object_detection import ObjectDetection

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))
                
def position_barrier(box, bar, pt, first):
    bar2 = bar[0]
    (ax,ay),(bx,by) = bar2
    
    if ay > by:
        x0,y0 = ax,ay
        x1,y1 = bx,by
    else: 
        x0,y0 = bx,by
        x1,y1 = ax,ay
    
    (x2,y2) = pt
    
    value = ((x1 - x0)*(y2 - y0) - (x2 - x0)*(y1 - y0))
    thresh = 1e-9
    
    if first == True:
        bot = (x0,y0-165)
        top = (x1,y1+165)
        bar_array = np.asarray([bot, top])
        dist = np.sum((bar_array-pt)**2, axis=1)
        near = np.argmin(dist)
        if box == 1:
            if value >= thresh:
                if near == 1:
                    return("Yes")
                else:
                    return("Wrong")
            elif value <= -thresh:
                return("No")
        elif box == 2:
            if value >= thresh:
                return("No")
            elif value <= -thresh:
                if near == 1:
                    return("Yes")
                else:
                    return("Wrong")
        elif box == 3:
            if value >= thresh:
                if near == 0:
                    return("Yes")
                else:
                    return("Wrong")
            elif value <= -thresh:
                return("No")
        elif box == 4:
            if value >= thresh:
                return("No")
            elif value <= -thresh:
                if near == 0:
                    return("Yes")
                else:
                    return("Wrong")
    else:
        if box == 1 or box == 3:
            if value >= thresh:
                return("Yes")
            elif value <= -thresh:
                return("No")
        elif box == 2 or box == 4:
            if value >= thresh:
                return("No")
            elif value <= -thresh:
                return("Yes")

video_list_path = list(clear_filepath("videos/to_analyse/detour"))
print(video_list_path)

od= ObjectDetection()

def dist(pt1, pt2):
    return(math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1]))

past1 = {}
past2 = {}
past3 = {}
past4 = {}
past5 = {}
past6 = {}
past7 = {}
past8 = {}
past9 = {}
past10 = {}
past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]

box_bounds = pd.read_csv("Boundary_coords.csv")

print(time.localtime()[3], ":", time.localtime()[4], ":", time.localtime()[5])

for video in video_list_path :
    
    file_name = os.path.basename(video)
    img_name = file_name.removesuffix(".mp4") + "_" + "track" + ".jpg"
    dest_img = ("detour/track_images/" + img_name)
    dest_backup = ("detour/track_images_clean/" + img_name)
    
    l = [idx for idx, item in enumerate(file_name.lower()) if "_" in item]
    start_pos = l[0] + 1
    end_pos = l[1]
    pos_name = file_name[start_pos:end_pos]
    print(pos_name)
    
    df = pd.DataFrame(columns=["Video", "Frame", "Box", "Fish", "x_coord", "y_coord", "Past_barrier", "Certainty"])
    
    df_check = pd.DataFrame(columns=["Video", "Frame", "Box"])
    
    print(file_name)
    cap = cv2.VideoCapture(video)
    
    camera_coords = box_bounds[box_bounds.Position == pos_name]
    
    box_coords=[]
    temp = list(camera_coords.Coordinates[camera_coords.Point == "box"])
    if len(temp) == 0:
        print(file_name, "missing boundaries")
        cap.release()
        continue
    
    for n in temp:
        n=n.replace("(","")
        n=n.replace(")","")
        n=n.split(",")
        n[0] = int(n[0])
        n[1] = int(n[1])
        box_coords.append(n)
    
    barrier=[]
    temp = list(camera_coords.Coordinates[camera_coords.Point == "barrier"])
    if len(temp) == 0:
        print(file_name, "missing boundaries")
        cap.release()
        continue
        
    for n in temp:
        n=n.replace("(","")
        n=n.replace(")","")
        n=n.split(",")
        n[0] = int(n[0])
        n[1] = int(n[1])
        barrier.append(n)
    
    test=[]
    for n in box_coords:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    bound_tl=test[0:4]
    bound_tr=test[4:8]
    bound_bl=test[8:12]
    bound_br=test[12:16]
    
    #Polygons for seeing if point within, array for showing on video
    polygon_tl=Polygon(bound_tl)
    polygon_tr=Polygon(bound_tr)
    polygon_bl=Polygon(bound_bl)
    polygon_br=Polygon(bound_br)
    
    polygon_dict = {1: (polygon_tl), 2: (polygon_tr), 3: (polygon_bl), 4: (polygon_br)}
    
    tl_array = np.array([bound_tl], np.int32)
    tr_array = np.array([bound_tr], np.int32)
    bl_array = np.array([bound_bl], np.int32)
    br_array = np.array([bound_br], np.int32)
    
    array_list = (tl_array, tr_array, bl_array, br_array)
    
    test=[]
    for n in barrier:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    bar1_coord=test[0:2]
    bar2_coord=test[2:4]
    bar3_coord=test[4:6]
    bar4_coord=test[6:8]
    
    bar1_array=np.array([bar1_coord], np.int32)
    bar2_array=np.array([bar2_coord], np.int32)
    bar3_array=np.array([bar3_coord], np.int32)
    bar4_array=np.array([bar4_coord], np.int32)
    
    bar_dict = {1: bar1_array, 2: bar2_array, 3: bar3_array, 4: bar4_array}
    
    coord_dict={1:(polygon_tl, bar1_array), 2:(polygon_tr, bar2_array), 3:(polygon_bl, bar3_array), 4:(polygon_br, bar4_array)}
    
    used_IDs = []
    
    count = 0
    track_id=1
    prev_detections=[]
    tracking_objects={}
    object_trail={}
    
    first_barrier = {1:True, 2:True, 3:True, 4:True}
    
    while True:
        
        ret, frame= cap.read()
        count = count + 1
        if not ret:
            break
        
        current_detections=[]
        tracking_objects_boxes={}
        
        (class_ids, scores, boxes) =od.detect(frame)
        box_nr = 0
        
        for box in boxes:
            (x,y,w,h)=box
            
            cx= x+w/2
            cy= y+h/2
            cv2.circle(frame,(int(cx),int(cy)),5,(0,0,120),-1)  
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,120),2)
            
            current_detections.append([round(scores[box_nr],2),(cx,cy)])
            box_nr+=1
        
        for container in array_list:
            cv2.polylines(frame, [container], True, (0,120,0))
        
        for key, item in coord_dict.items():
            (box, bar) = item
            for barrier in bar:
                cv2.polylines(frame, [barrier], True, (0,0,120))
        
        if count <=2:
            for score, pt in current_detections:
                for score2, pt2 in prev_detections:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[track_id] = [score,pt]
                        used_IDs.append(track_id)
                        track_id = track_id + 1
                        for key, item in coord_dict.items():
                            box,bar = item
                            if box.contains(Point(pt)) == True:
                                if box in tracking_objects_boxes.keys():
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [file_name], "Frame": [count], 
                                                                                 "Box": [key]})], ignore_index=True)
                                else:
                                    tracking_objects_boxes[box] = pt
                        
                            
        else:
            tracking_objects_copy = tracking_objects.copy()
            current_detections_copy = current_detections.copy()
            
            for object_id, box in tracking_objects_copy.items():
                pt2 = box[1]
                object_exists = False
                for score, pt in current_detections_copy:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[object_id]= [score,pt]
                        object_exists = True
                        for key, item in coord_dict.items():
                            box,bar = item
                            if box.contains(Point(pt)) == True:
                                if box in tracking_objects_boxes.keys():
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [file_name], "Frame": [count], 
                                                                                 "Box": [key]})], ignore_index=True)
                                else:
                                    tracking_objects_boxes[box] = pt
                        if [score, pt] in current_detections:
                            current_detections.remove([score, pt])
                        continue
                
                if not object_exists:
                    past1[object_id] = tracking_objects[object_id]
                    tracking_objects.pop(object_id)
            
            for score, pt in current_detections:
                found = False
                for past in past_list:
                    past_copy = past.copy()
                    if found == True:
                        break
                    
                    for object_id, box in past_copy.items():
                        pt2 = box[1]
                        distance = dist(pt, pt2)
                        
                        if distance < 50:
                            tracking_objects.update({object_id: [score,pt]})
                            found = True
                            past.pop(object_id)
                            continue
                        
                        if found == True:
                            break
                
                if found == False:
                    tracking_objects[track_id] = [score,pt]
                    used_IDs.append(track_id)
                    track_id = track_id + 1
                    for key, item in coord_dict.items():
                            box,bar = item
                            if box.contains(Point(pt)) == True:
                                if box in tracking_objects_boxes.keys():
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [file_name], "Frame": [count], 
                                                                                 "Box": [key]})], ignore_index=True)
                                else:
                                    tracking_objects_boxes[box] = pt
        
        
        past10 = past9
        past9 = past8
        past8 = past7
        past7 = past6
        past6 = past5
        past5 = past4
        past4 = past3
        past3 = past2
        past2 = past1
        past1 = {}
        past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]
        
        track_objects_clear = []
        
        box_check = {}
        for object_id, box in tracking_objects.items():
            score = box[0]
            pt = box[1]
            cv2.putText(frame,str((object_id, score)),(int(pt[0]),int(pt[1])-7),0,1,(0,120,120),2)
            
            for key, item in coord_dict.items():
                box,bar = item
                box_nr = 0 # this is so that if outside box has nr 0
                if box.contains(Point(pt)) == True:
                    box_nr = key
                    past_barrier = position_barrier(key, bar, pt, first_barrier[key])
                    temp = [object_id, box_nr, pt[0], pt[1], past_barrier, score]
                    if first_barrier[key] == True:
                        if past_barrier != "No":
                            first_barrier[key] = False
                    elif first_barrier[key] == False:
                        if past_barrier == "No":
                            first_barrier[key] == True
                    track_objects_clear.append(temp)
                    if box_nr in object_trail.keys():
                        object_trail[box_nr].append(pt)
                        box_check.update({box_nr:"old"})
                    else:
                        object_trail.update({box_nr:[pt]})
                        box_check.update({box_nr:"new"})
                    break
        
        print("Frame", count)
        
        df_coords = pd.DataFrame(track_objects_clear, columns = ["Fish", "Box", "x_coord", "y_coord", "Past_barrier", "Certainty"])
        df_coords["Frame"] = count
        df_coords["Video"] = file_name
        df = pd.concat([df, df_coords], ignore_index=True)
        
        cv2.namedWindow("Fish", cv2.WINDOW_NORMAL)
        cv2.imshow("Fish",frame)
        
        prev_detections= current_detections.copy()
        
        if count == int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
            cv2.imwrite(dest_backup, frame)
            for box_id, pt_list in object_trail.items():
                if len(pt_list) > 1:
                    pts = np.array(pt_list, np.int32)
                    cv2.polylines(frame,[pts],False,(0,255,255))
                    
            cv2.imwrite(dest_img, frame)
            
        
        key= cv2.waitKey(1)
        if key == 27:
            break
    
    cap.release()
    
    if os.path.exists("detour/Detour_data.csv"):
        df.to_csv("detour/Detour_data.csv", index=False, header=False, mode="a")
    else:
        df.to_csv("detour/Detour_data.csv", index=False)
    dest_done = ("videos/analysed/detour/" + os.path.basename(video))
    shutil.move(video, dest_done)
    
    if len(df_check) > 0:
        if os.path.exists("detour/Dupes_in_boxes.csv"):
            df_check.to_csv("detour/Dupes_in_boxes.csv", index=False, header=False, mode="a")
        else:
            df_check.to_csv("detour/Dupes_in_boxes.csv", index=False)
    
    print(time.localtime()[3], ":", time.localtime()[4], ":", time.localtime()[5])
    
    cv2.destroyAllWindows()


print("done")